<a href="https://colab.research.google.com/github/jaswanthreddy049/a2z-strivers/blob/main/GenAIcohort_May2025_PromptEng_jaswanth_reddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ASSIGNMENT_1



In [25]:
!pip install -q groq litellm langchain_groq

In [26]:
import os
from google.colab import userdata
from groq import Groq
from langchain_groq import ChatGroq
from litellm import completion

In [27]:
groq_api_key = userdata.get('demo')
os.environ["GROQ_API_KEY"] = groq_api_key
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [28]:
def get_response(prompt):
  #define model
  model="groq/llama3-70b-8192"
  messages = [{"role": "user", "content": prompt}]

  #call responses
  response = completion(
      model=model,
      messages=messages
  )

  return response['choices'][0]['message']['content'] or ""


In [29]:
text=f"""Black..... holes—those mysterious cosmic phenomena—are regions in space with gravitational forces so intense that not even light can escape. Astrophysicists Dr. Alan Rothwell and Prof. Priya Sinha### have dived deeply into the “science”

of these objects, uncovering fascinating insights! According to Rothwell’s research (2021), black holes form when massive stars burn ^^^^through their fuel & collapse under gravity until they reach a singularity—a point of (nearly) infinite density, where physics bReak dOwn.

Sinha’s studies focus on the event horizon, or the “point of no return”; any matter or light crossing this boundary is trapped forever. Sinha also examines Hawking Radiation—a theoretical ....concept from Stephen Hawking—suggesting that black holes.... emit small amounts of energy...., slowly losing mass. Such radiation might allow scientists to peek inside these intense entities!!!!. Together, Rothwell and Sinha’s studies hint at bridging quantum mechanics w/ general relativity—fields otherwise tough to reconcile!

Remember to clean the text."""

prompt=f"""
        you will bw provided with a text in between triple codes text is ''' {text}''',summaries the given information in 100 words the text should be clean.
        every line should be of equal length"""
response=get_response(prompt)
print(response)

Here is a 100-word summary of the text:

Black holes are regions in space with intense gravitational forces that not even light can escape. Astrophysicists Dr. Alan Rothwell and Prof. Priya Sinha have delved into the science of these objects. Rothwell's research shows that black holes form when massive stars collapse under gravity, reaching a singularity. Sinha's studies focus on the event horizon, where matter and light are trapped, and Hawking Radiation suggests black holes emit energy, slowly losing mass. Their studies hint at bridging quantum mechanics and general relativity, fields tough to reconcile. The research provides fascinating insights into the mysterious nature of black holes.


In [30]:
review=f"""I absolutely loved the new update—it’s so user-friendly and fast!"""
prompt=f"""
      you will be given with an example review and your task is to classify the review as positive or negative or neutral.
      here is the review '''{review}'''."""
response=get_response(prompt)
print(response)


Based on the language used in the review, I would classify it as **POSITIVE**. The words "absolutely loved", "user-friendly", and "fast" all have positive connotations, indicating that the reviewer is very happy with the new update.


In [31]:
userreq=f"""get me candidates with 2 years experience in python and flask and candidate should academic grades should more than 70 percentage ready to work in hyderabad"""
prompt=f"""generate a mangodb query in pymango formate to filter the resumes with this query'''{userreq}'''"""
response=get_response(prompt)
print(response)

Here is the query in PyMongo format:
```
query = {
    "$and": [
        {"experience": {"$gte": 2}},
        {"skills": {"$all": ["python", "flask"]}},
        {"academic_grades": {"$gt": 70}},
        {"ready_to_work_in": "hyderabad"}
    ]
}
```
Here's a breakdown of the query:

* `"$and"`: This is a logical operator to combine multiple filter clauses.
* `{"experience": {"$gte": 2}}`: This clause filters candidates with at least 2 years of experience. The `$gte` operator is used to specify a minimum value.
* `{"skills": {"$all": ["python", "flask"]}}`: This clause filters candidates who have both "python" and "flask" skills. The `$all` operator is used to specify an array of values that must be present in the `skills` field.
* `{"academic_grades": {"$gt": 70}}`: This clause filters candidates with academic grades greater than 70%. The `$gt` operator is used to specify a minimum value.
* `{"ready_to_work_in": "hyderabad"}`: This clause filters candidates who are ready to work in Hyde

ASSIGNMENT 2

In [32]:
!pip install -q groq litellm langchain-groq langchain langchain-community langgraph

In [33]:
import os
from google.colab import userdata
groq_api_key = userdata.get('demo')
os.environ["GROQ_API_KEY"] = groq_api_key

In [34]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from langgraph.prebuilt import create_react_agent
from typing import List
from litellm import completion
from langchain.agents import initialize_agent, Tool, AgentType

In [35]:
# from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile", #"llama-3.1-8b-instant",
    temperature=0, # range of temperature variable is from 0(least artistic) to 1(most artistic/enhanced)
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [36]:
# from typing import List
class emotionalOutput(BaseModel):
    """
    Defines the structured output schema that the LLM must follow.

    Fields:
    - Patient's emotional state (e.g., fear, anxious, calm)
    - score: emotion intensity score from 1 to 5 as an integer
    """
    sentiment: str = Field(description="emotion of the patient in the conversation")
    score: int = Field(description="patient emotional intensity score  on a scale of 1 to 5")

class Communicationsummary(BaseModel):
    """
    Defines the structured output schema that the LLM must follow.

    Fields:
    - summary :List of summaries and urgency assessment
    """
    improvements: List[str] = Field(description="List of summarie of the conversation and urgency oof the patient")


In [37]:
from langchain.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=emotionalOutput)
@tool
def analyze_patient_emotion(conversation: str):
    """
    Analyzes a patient-agent conversation to extract:
    1. The patients emotion (e.g.,cool,fear,anxious etc..)
    2. A patients emotion score (1 to 5)

    Parameters:
    - conversation (str): The full conversation between the agent and the patient.

    Returns:
    - emtionOutput: Contains emotion and score, parsed and validated by the Pydantic model.
    """

    # Prompt tells the LLM exactly what to do and restricts it from returning anything extra.
    prompt = f"""
    You are an expert in analyzing healthcare conversations.

    Analyze the given conversation {conversation} between a agent and a patient and extract the following:
    1. patient emotion (clam,fear, anxious, etc.)
    2. Emotional intensity score (1 to 5)
    3. Short summary of patient complaint and urgency level
    Strictly respond following the below schema:
    {parser.get_format_instructions()}
    """

    #using chatgroq from langchain
    response = llm.invoke(prompt)
    cleaned_output = response.content.strip()
    return cleaned_output


In [38]:
new_tools = [
    Tool(
        name="emotionAnalysisTool",
        func=analyze_patient_emotion,
        description="""Use this tool to analyze the patient emotion (e.g.,fear,calm,anxious) and
        give a emotion intensity score (1–5). """
    ),
]

#Bind the LLM with your tools
llm_with_new_tools = llm.bind_tools(new_tools)

In [39]:
agent_executor = create_react_agent(llm_with_new_tools, new_tools)

In [40]:
conversation = """
Agent: Good morning! Thank you for reaching out to CareConnect. How can I assist you today?
Patient: I’ve been having chest pain for the past two days, and it’s getting worse.
Agent: I’m really sorry to hear that. Let’s look into it right away. Can you describe the pain a bit more?
Patient: This is the second time in a month this has happened. It’s really worrying and exhausting.
Agent: I understand your concern. I’ll schedule a priority consultation and flag this for immediate review.
Patient: Okay. Please make sure this gets taken seriously this time.
"""

In [41]:
response = agent_executor.invoke({"messages": [HumanMessage(content=f"what is customer emotion intensity score in {conversation}?")]})
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

The patient's emotion intensity score is 4, with the dominant emotion being anxious.
